In [2]:
# upload the kaggle.json
from  google.colab import files
files.upload()

Saving kaggle.json to kaggle (1).json


{'kaggle.json': b'{"username":"jaynakum","key":"dc302c8852ddebe9f92617524a91e171"}'}

In [3]:
# install the kaggle package -q is to ignore the console logs
!pip install -q kaggle

In [4]:
# create a .kaggle folder
!mkdir -p ~/.kaggle

In [5]:
# copy the json file
!cp kaggle.json ~/.kaggle

In [6]:
# give the read and write permissions
!chmod 600 ~/.kaggle/kaggle.json

In [7]:
!mkdir dogbreed

In [8]:
%cd dogbreed

/content/dogbreed


In [9]:
!kaggle datasets download -d catherinehorng/dogbreedidfromcomp

 99% 687M/691M [00:04<00:00, 166MB/s]
100% 691M/691M [00:04<00:00, 176MB/s]


In [ ]:
!unzip dogbreedidfromcomp.zip

In [11]:
!rm sample_submission.csv

In [54]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import label_binarize

In [55]:
all_labels = pd.read_csv('/content/dogbreed/labels.csv')
all_labels.head()

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo
2,001cdf01b096e06d78e9e5112d419397,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever


In [56]:
all_labels['breed'].value_counts()

scottish_deerhound      126
maltese_dog             117
afghan_hound            116
entlebucher             115
bernese_mountain_dog    114
                       ... 
golden_retriever         67
brabancon_griffon        67
komondor                 67
eskimo_dog               66
briard                   66
Name: breed, Length: 120, dtype: int64

In [57]:
class_name = ["scottish_deerhound", "maltese_dog", "bernese_mountain_dog"]
labels = all_labels[(all_labels['breed'].isin(class_name))]
labels = labels.reset_index()
labels.head()

,index,id,breed
0,9,0042188c895a2f14ef64a918ed9c7b64,scottish_deerhound
1,12,00693b8bc2470375cc744a6391d397ec,maltese_dog
2,79,01e787576c003930f96c966f9c3e1d44,scottish_deerhound
3,90,022b34fd8734b39995a9f38a4f3e7b6b,maltese_dog
4,118,02d54f0dfb40038765e838459ae8c956,bernese_mountain_dog


In [58]:
X_data = np.zeros((len(labels), 224, 224, 3), dtype='float32')
Y_data = label_binarize(labels['breed'], classes=class_name)
X_data.shape, Y_data.shape

((357, 224, 224, 3), (357, 3))

In [59]:
import cv2

from keras.utils import img_to_array

In [60]:
for i in range(len(labels)):
  img = cv2.imread('/content/dogbreed/train/%s.jpg' % labels['id'][i])
  img = img_to_array(img)
  img = cv2.resize(img, (224, 224))
  
  x = np.expand_dims(img.copy(), axis=0)
  X_data[i] = x / 255.0
X_data.shape

(357, 224, 224, 3)

In [61]:
from sklearn.model_selection import train_test_split

In [62]:
xtrain, xtest, ytrain, ytest = train_test_split(X_data, Y_data, random_state=0)
xnew_train, xvalid, ynew_train, yvalid = train_test_split(xtrain, ytrain)

In [63]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten
from keras.optimizers import Adam

In [64]:
model = Sequential()
model.add(Conv2D(32, (3,3), input_shape=(224, 224, 3), activation='relu', kernel_regularizer='l2')) # L2
model.add(Conv2D(32, (3,3), activation='relu', kernel_regularizer='l2'))
model.add(MaxPool2D(pool_size=(3,3)))
model.add(Conv2D(32, (3,3), activation='relu', kernel_regularizer='l2'))
model.add(MaxPool2D(pool_size=(3,3)))
model.add(Conv2D(32, (3,3), activation='relu', kernel_regularizer='l2'))
model.add(MaxPool2D(pool_size=(3,3)))
model.add(Flatten())
model.add(Dense(30, activation='relu'))
model.add(Dense(3, activation='softmax'))
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8 (Conv2D)           (None, 222, 222, 32)      896       
                                                                 
 conv2d_9 (Conv2D)           (None, 220, 220, 32)      9248      
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 73, 73, 32)       0         
 2D)                                                             
                                                                 
 conv2d_10 (Conv2D)          (None, 71, 71, 32)        9248      
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 23, 23, 32)       0         
 2D)                                                             
                                                                 
 conv2d_11 (Conv2D)          (None, 21, 21, 32)       

In [65]:
model.compile(loss='categorical_crossentropy', optimizer=Adam(0.001), metrics=['accuracy'])

In [66]:
hist = model.fit(xnew_train, ynew_train, epochs=2, batch_size=32, validation_data=(xvalid, yvalid))

Epoch 1/2
7/7 [==============================] - 14s 310ms/step - loss: 2.0643 - accuracy: 0.3350 - val_loss: 1.9508 - val_accuracy: 0.4030
Epoch 2/2
7/7 [==============================] - 0s 69ms/step - loss: 1.9069 - accuracy: 0.3750 - val_loss: 1.8088 - val_accuracy: 0.4030


In [67]:
model.evaluate(xtest, ytest)

3/3 [==============================] - 1s 391ms/step - loss: 1.8243 - accuracy: 0.2889


[1.8242732286453247, 0.2888889014720917]

In [74]:
ypred = model.predict(xtest)

3/3 [==============================] - 0s 23ms/step


In [79]:
from sklearn.metrics import classification_report

In [80]:
pred = np.argmax(ypred, axis=1)
actual = np.argmax(ytest, axis=1)

In [83]:
print(classification_report(actual, pred))

              precision    recall  f1-score   support

           0       0.29      1.00      0.45        26
           1       0.00      0.00      0.00        30
           2       0.00      0.00      0.00        34

    accuracy                           0.29        90
   macro avg       0.10      0.33      0.15        90
weighted avg       0.08      0.29      0.13        90



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [84]:
model.save('dogbreed.h5')